# Exercise 5

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import histeq_modified as hm

FS = 15 # Fontsize of caption

## 5.1 Global Histogram Equalization

In [ ]:
# Load the image (grayscale)
image = cv2.imread('moon.jpg', cv2.IMREAD_GRAYSCALE)

# Display the original image and its histogram
plt.figure(figsize=(12, 6))

# Original image
plt.subplot(2, 2, 1)
plt.imshow(image, cmap='gray')
plt.title("Original Image", fontsize = FS)
plt.axis('off')

# Histogram of the original image
plt.subplot(2, 2, 2)
hist, bins = np.histogram(image.flatten(), 256, [0, 256])
plt.bar(bins[:-1], hist, width=1, color='gray', edgecolor='black')
plt.title("Histogram Before Equalization", fontsize = FS)
plt.xlabel("Gray Level")
plt.ylabel("% of Pixels")
plt.grid()

# Apply histogram equalization
equalized_image = cv2.equalizeHist(image)

# Equalized image
plt.subplot(2, 2, 3)
plt.imshow(equalized_image, cmap='gray')
plt.title("Equalized Image", fontsize = FS)
plt.axis('off')

# Histogram of the equalized image
plt.subplot(2, 2, 4)
hist_eq, bins_eq = np.histogram(equalized_image.flatten(), 256, [0, 256])
plt.bar(bins_eq[:-1], hist_eq, width=1, color='gray', edgecolor='black')
plt.title("Histogram After Equalization", fontsize = FS)
plt.xlabel("Gray Level")
plt.ylabel("% of Pixels")
plt.grid()

# Show the results
plt.tight_layout()
plt.show()

## 5.2 Contrast Limited Adaptive Histogram Equalization (CLAHE)

In [ ]:
# Load image
image = cv2.imread('moon.jpg', cv2.IMREAD_GRAYSCALE)

# Parameters
clip_ratios = [1.0, 0.7, 0.4, 0.1]  # Clipping ratios

# Compute original histogram
hist, bins = np.histogram(image.flatten(), 256, [0, 256])
max_count = hist.max()

# Prepare results storage
limited_eq_images = []
LUT = np.zeros((len(clip_ratios), 256), dtype=np.uint8)

# Loop over different clipping ratios
for i, clip_ratio in enumerate(clip_ratios):
    clip_value = clip_ratio * max_count
    # Clip histogram
    clipped_hist = np.clip(hist, 0, clip_value)

    # Generate virtual pixel values for histogram equalization
    clipped_values = []
    for level in range(256):
        clipped_values.extend([level] * int(clipped_hist[level]))
    
    # Create mapping function via histogram equalization
    clipped_values = np.array(clipped_values, dtype=np.uint8)
    temp, mapping = cv2.calcHist([clipped_values], [0], None, [256], [0, 256]), None
    T = np.cumsum(clipped_hist) / np.cumsum(clipped_hist)[-1]
    T = np.clip(T, 0, 1)
    
    LUT[i, :] = (T * 255).astype(np.uint8)

    # Apply mapping to the original image
    limited_eq_images.append(cv2.LUT(image, LUT[i, :]))

# Display results
plt.figure(figsize=(12, 6))
for i, eq_img in enumerate(limited_eq_images):
    plt.subplot(1, len(clip_ratios), i + 1)
    plt.imshow(eq_img, cmap='gray')
    plt.title(f'Clip Ratio: {clip_ratios[i]}', fontsize=FS)
    plt.axis('off')

plt.figure(figsize=(12, 6))
# Original histogram
plt.subplot(1, 2, 1)
plt.bar(bins[:-1], hist, width=1, color='gray', edgecolor='black')
plt.title('Original Histogram', fontsize=FS)
plt.xlabel('Gray Level', fontsize=FS)
plt.ylabel('Pixel Count', fontsize=FS)
plt.grid()

# LUTs
plt.subplot(1, 2, 2)
for i, lut in enumerate(LUT):
    plt.plot(lut, label=f'Clip Ratio: {clip_ratios[i]}', linewidth=2)
plt.title('Mapping Functions (LUT)', fontsize=FS)
plt.xlabel('Input Intensity', fontsize=FS)
plt.ylabel('Output Intensity', fontsize=FS)
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


## 5.3 Adaptive Histogram Equalization

In [ ]:
# Load the image
I = cv2.imread('parrot.jpg', cv2.IMREAD_GRAYSCALE)

# Perform global histogram equalization
eq_I = cv2.equalizeHist(I)

# Perform local histogram equalization (using blocks)
def local_hist_eq(img, block_size = (200, 200)):
    rows, cols = img.shape
    out_img = np.zeros_like(img)
    
    # Define the block size
    block_h, block_w = block_size
    
    # Iterate over the image in blocks
    for i in range(0, rows, block_h):
        for j in range(0, cols, block_w):
            block = img[i:i+block_h, j:j+block_w]
            out_img[i:i+block_h, j:j+block_w] = cv2.equalizeHist(block)
    
    return out_img

lc_I = local_hist_eq(I)

# Display the images
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original image', fontsize = FS)

plt.subplot(1, 3, 2)
plt.imshow(eq_I, cmap = 'gray')
plt.title('Global Equalization', fontsize = FS)

plt.subplot(1, 3, 3)
plt.imshow(lc_I, cmap = 'gray')
plt.title('Local Equalization', fontsize = FS)

plt.show()

# Plot histograms
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Histogram before equalization
count, bins = np.histogram(I.flatten(), bins = 256, range = [0, 255])
ax[0].bar(bins[:-1], count, width = 1)
ax[0].set_title('Histogram before equalization', fontsize = FS)
ax[0].set_xlim([0, 255])
ax[0].set_xlabel('Gray level')
ax[0].set_ylabel('% of pixels')

# Histogram after global equalization
count, bins = np.histogram(eq_I.flatten(), bins = 256, range = [0, 255])
ax[1].bar(bins[:-1], count, width=1)
ax[1].set_title('Histogram after global equalization', fontsize = FS)
ax[1].set_xlim([0, 255])
ax[1].set_xlabel('Gray level')
ax[1].set_ylabel('% of pixels')

# Histogram after local equalization
count, bins = np.histogram(lc_I.flatten(), bins = 256, range = [0, 255])
ax[2].bar(bins[:-1], count, width=1)
ax[2].set_title('Histogram after local equalization', fontsize = FS)
ax[2].set_xlim([0, 255])
ax[2].set_xlabel('Gray level')
ax[2].set_ylabel('% of pixels')

plt.show()

## 5.4 Convolution between an image and a simple filter

In [ ]:
# Load the image
I = cv2.imread('airplane.png', cv2.IMREAD_GRAYSCALE)

# Constract three different filters
h1 = np.ones((1, 10)) / 10 # horizontal filter
h2 = np.ones((10, 1)) / 10 # vertical filter
h3 = np.ones((10, 10)) / 100 # box/windown filter

# Perform filtering on the noisy image
filtered_I1 = cv2.filter2D(I, -1, h1)
filtered_I2 = cv2.filter2D(I, -1, h2)
filtered_I3 = cv2.filter2D(I, -1, h3)

# Show the images
plt.figure(figsize=(10, 5))

# Horizontal filtering
plt.subplot(1, 2, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original image', fontsize = FS)

plt.subplot(1, 2, 2)
plt.imshow(filtered_I1, cmap = 'gray')
plt.title('Horizontal filtering', fontsize = FS)

plt.show()

# Show Vertical filtering
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original image', fontsize = FS)

plt.subplot(1, 2, 2)
plt.imshow(filtered_I2, cmap = 'gray')
plt.title('Vertical filtering', fontsize = FS)

plt.show()

# Show Box filtering
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original image', fontsize = FS)

plt.subplot(1, 2, 2)
plt.imshow(filtered_I3, cmap = 'gray')
plt.title('Box filtering', fontsize = FS)

plt.show()

## 5.5 Subsampling

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
I = cv2.imread('bike.png', cv2.IMREAD_GRAYSCALE)

# Normalize the image to 0-1 (similar to im2double in MATLAB)
I = I.astype(np.float32) / 255.0

# Get the dimensions of the image
row, column = I.shape

# Perform subsampling by matrix multiplication
# Create the matrices h1 and h2
a = np.eye(row // 2)  # Identity matrix
b = np.array([1, 0])  # Vector for horizontal subsampling
c = np.array([0.5, 0.5])  # Vector for combined subsampling

h1 = np.kron(a, b)  # Kronecker product
h2 = np.kron(a, c)  # Kronecker product

# Perform subsampling
# sub_I1 = h1.T @ I @ h1  # h1.T: transpose of h1
# sub_I2 = h2.T @ I @ h2

# Perform subsampling by taking every second row and column
sub_I1 = I[::2, ::2]  # Method 1: Take every second row and column

# Method 2: A more advanced subsampling (weighted)
sub_I2 = (I[::2, ::2] + I[1::2, ::2] + I[::2, 1::2] + I[1::2, 1::2]) / 4  # Averaging 4 neighboring pixels

# Show images
plt.figure(figsize=(15, 5))

# Original image, Method 1, and Method 2
plt.subplot(1, 3, 1)
plt.imshow(I, cmap = 'gray')
plt.title('Original image', fontsize = FS)

plt.subplot(1, 3, 2)
plt.imshow(sub_I1, cmap = 'gray')
plt.title('Method 1', fontsize = FS)

plt.subplot(1, 3, 3)
plt.imshow(sub_I2, cmap = 'gray')
plt.title('Method 2', fontsize = FS)

plt.show()